### 三、一个简单Constraints的例子

这里我们将使用ROSETTA中的constraint file（限制文件）。

限制文件中按照一定格式规定限制类型、限制对象、限制的惩罚函数形式。

    -> 限制文件的格式：参考 https://new.rosettacommons.org/docs/latest/rosetta_basics/file_types/constraint-file
    
    -> 作者已经预先在data文件目录下准备了constraint_file。

! under linux shell

cat constraint_file

AtomPair CA 20 CA 6 LINEAR_PENALTY 9.0 0 0 1.0

In [15]:
# 在1ubq_clean.pdb的例子上施加原子对的限制

from pyrosetta import *
pyrosetta.init()

# 初始化REF2015的打分函数对象
my_scorefxn = create_score_function('ref2015')

# reweight the score I need
# 需要先将原子对的所属定义的score项权重设定为1
my_scorefxn.set_weight(pyrosetta.rosetta.core.scoring.atom_pair_constraint, 1.0) 

PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python36.ubuntu 2020.28+release.8ecab77aa50ac1301efe53641e07e09ac91fee3b 2020-07-07T16:41:06] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.ubuntu r260 2020.28+release.8ecab77aa50 8ecab77aa50ac1301efe53641e07e09ac91fee3b http://www.pyrosetta.org 2020-07-07T16:41:06
core.init: command: PyRosetta -ex1 -ex2aro -database /home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1076598603 seed_offset=0 real_seed=1076598603
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1076598603 RG_type=mt19937


In [16]:
from pyrosetta.rosetta.protocols.constraint_movers import *
cst_set = ConstraintSetMover()
cst_set.add_constraints(False) # True=在原有限制基础上额外再添加限制，False= 从文件中读取并覆盖所有的限制。
cst_set.constraint_file('./data/constraint_file')

In [17]:
# load pose from 1ubq_clean.pdb
initial_pose = pose_from_pdb("./data/1ubq_clean.pdb")

pose = Pose()
pose.assign(initial_pose)
cst_set.apply(pose)

core.import_pose.import_pose: File './data/1ubq_clean.pdb' automatically determined to be of type PDB
core.scoring.constraints.ConstraintsIO: read constraints from ./data/constraint_file
core.scoring.constraints.ConstraintsIO: Read in 1 constraints


In [18]:
# 打印查看当前pose所有设定的constraint
print(pose.constraint_set())

ResiduePairConstraints: total: 20   plotting active...
ResiduePairConstraints (6,20)
AtomPairConstraint (2,20-2,6)
         r      func     dfunc dfunc_est
         2     7.000     0.000    -1.000
       2.5     6.500     0.000    -1.000
         3     6.000     0.000    -1.000
       3.5     5.500     0.000    -1.000
         4     5.000     0.000    -1.000
       4.5     4.500     0.000    -1.000
         5     4.000     0.000    -1.000
       5.5     3.500     0.000    -1.000
         6     3.000     0.000    -1.000
       6.5     2.500     0.000    -1.000
         7     2.000     0.000    -1.000
       7.5     1.500     0.000    -1.000
         8     1.000     0.000    -1.000
       8.5     0.500     0.000    -1.000
         9     0.000     0.000     0.000
       9.5     0.500     0.000     1.000
        10     1.000     0.000     1.000
      10.5     1.500     0.000     1.000
        11     2.000     0.000     1.000
      11.5     2.500     0.000     1.000
        12     3.000    

### 小结

从这个简单的例子中可以看到，使用“限制文件”对pose进行限制的方法理论上适用于所有的场景。因为只要我们按照rosetta的要求编写所有的限制规则于限制文件中，这种方法就可以最大化用户自由度，但是缺点是使用“限制文件”事先需要对限制文件的编写规则有一定的了解和把握。



### 习题

1. 在1ubq_clean.pdb的pose中使用HARMONIC限制某二面角（可自己选择）。（注意score需要实现设定对应weight为1的开放状态；难度：*）